## Monte Carlo Methods

- <font color=deeppink>MC是做均值估计很常见的方法，而$V(s)和Q(s,a)$都是以均值形式定义的函数，因此可以用MC方法来估计state value和action value.</font>

### I. 最简单的MC-based 算法：direct learning

#### I.1 思路

1. 根据给定策略条件下的value function定义：
$$\begin{align} 
V^{\pi}(s)& =E(U|\pi,s_0=s)\\
& =E\left [ \sum_{t=0}^{\infty } \gamma ^tR(s_t, \pi(s_t), s_{t+1}) |s_0=s\right ] \\
\end{align}$$可以直接用抽样方法找不同states的$V^{\pi}(s)$估计值。

2. 方法：固定策略$\pi$，直接采样$V^{\pi}(s)$。1个以$s_i$为起点的episode得到1个$V^{\pi}(s_i)$的样本。用均值作为$V^{\pi}(s_i)$的估计值。
$$\begin{align} 
& sample_1 = V^{\pi}(s_i)=R(s,\pi(s),s^{'}_1)+\sum_{t=1}^{terminal } \gamma ^tR(s_{1,t}, \pi(s_{1,t}), s_{1,t+1})\\
& sample_2 = V^{\pi}(s_i)=R(s,\pi(s),s^{'}_2)+\sum_{t=1}^{terminal } \gamma ^tR(s_{2,t}, \pi(s_{2,t}), s_{2,t+1})\\
& ...\\
& sample_n = V^{\pi}(s_i)=R(s,\pi(s),s^{'}_n)+\sum_{t=1}^{terminal } \gamma ^tR(s_{n,t}, \pi(s_{n,t}), s_{n,t+1})\\
& V^{\pi }(s_i) \leftarrow \frac{1}{n}\sum_{j}sample_j 
\end{align}$$

3. 同样可以直接用抽样方法找不同states-action的$Q^{\pi}(s,a)$估计值。

#### I.2 算法

- <img src="./pics/MC_basics.png" alt="alt text" width="560"/>

#### I.3 应用要点

1. **用法**：
   - <font color=green>direct learning（包括后面的temporal difference learning）只是找$V^{\pi}$的方法。但原本的目标是找到最优策略。所以找到$V^{\pi}$之后，通常需要再结合model-based learning方法来获得transition function和rewards function的信息。有了$V^{\pi}(s),T(s, \pi(s), s^{'}),R(s, \pi(s), s^{'})$的信息后，再用
$$Q^*(s, a)=\sum_{s^{'}}^{}T(s,a,s^{'})[R(s,a,s^{'})+\gamma V^{*}(s^{'})]$$
来迭代最优策略。所以direct learning和TD learning相当于完成了用policy iteration求解MDP问题时的policy evaluation的那一步，而policy improvement/extraction的那一步还需要额外的model-based learning的方法来做。</font>
   - 但如果是直接抽样得到$Q^{\pi}(s,a)$那么policy improvement就很好做。但此时抽样的量会变大。[上面的算法中就是直接抽样Q而不是抽V]

2. 问题：
   - low sample efficiency：准确估计Value function所需要的episode量很大。因为1个以$s_i$为起点的episode只采纳对应的total rewards作为有效信息。这个episode在exploration过程中rewards value包含了不同的states之间transition的关联信息，都没有被用到。
   - 有的状态不可重现，这时候抽样很难

### II. MC exploring starts

#### 1. 思路
- direct learning方法只用了total rewards value，没有使用中间状态的reward信息，因此造成了信息浪费，一种解决这个问题的思路是，利用中间信息来做policy evaluation。<font color=red>不再用episode为抽样的单位，而是用一次action-reward为抽样的单位。</font>

#### 2. 方法
   - 假如有一个按照给定策略$\pi$抽样得到的episode数据: <font color=brown>注，下标是index of state/action，而不是index of time steps。</font>
$$s_1\overset{a_2}{\rightarrow } s_2\overset{a_4}{\rightarrow } s_1\overset{a_2}{\rightarrow } s_2\overset{a_3}{\rightarrow } s_5\overset{a_1}{\rightarrow }... $$
   - 这个episodes中每个state-action pair及其之后的trajectory都视为一个新的episode。
     - <img src="./pics/mc_exploring_example.png" alt="alt text" width="560"/>
   - If an episode is sufficiently long such that it can visit all the state-action pairs many times, then this single episode may be sufficient for estimating all the action values using the every-visit
strategy.

#### 3. 两种使用episodes的方式
1. <font color=blue>**first-visit strategy**</font>：每个state-action pair只用它第一次出现时的episode，后面再出现就不再作为新的episode。
   - 比如上例中的$(s_1, a_2)$出现了两次，但是只使用original episode作为它的sample
3. <font color=blue>**every-visit strategy**</font>：每个state-action pair每次出现位置到结尾的trajectory记为不同的episodes。<font color=green>此时sample efficiency最高</font>
   - 比如上例中的$(s_1, a_2)$，出现了两次，两次对应的两个episodes都作为它的samples
   - <font color=orange>**问题**：使用every-visit strategy得到的样本是相关的，不满足MC均值估计所需的iid条件。 因为，state-action第二次出现后的trajectory实际上是它第一次出现得到的episode的子集。不过，如果这两次出现的间隔足够长的话，相关性不会太强。</font>

#### 4. 两种计算action value的方式
1. 在policy evaluation步中，在当前策略下，先收集以每个(s,a) pair为起点的所有episodes数据，然后一次性求均值得到Q(s,a)。这也是MC basic中使用的方法。
   - 问题：必须等到所有episode都收集完之后再更新策略
3. 每得到一个episode就计算一次。相当于求均值的增量更新方法。这种方法的时间和空间复杂度都是$O(1)$，是更高效的方式。

#### 5. 算法伪码
   - 要点：
     1. 使用every-visit strategy
     2. Q(s,a)的均值计算采用增量更新方法
     3. 每个episode的信息都是从后向前累积得到reward
   - 伪码：
     - <img src="./pics/MC_exploring.png" alt="alt text" width="560"/>

3. 方法的条件限制
   - <font color=blue>**exploring-starts condition**</font>：指需要sufficiently many episodes starting from every state-action pair. 
     - Only if every state-action pair is well explored, can we accurately estimate their action values (according to the law of large numbers) and hence successfully find optimal policies.
     - MC Basic和MC Exploring Starts都需要满足这个条件，但实际环境中很可能难以精确控制这些因素。比如，状态难以复现的情况很常见。<font color=red>**这也是这两个算法在实践中很少被用到的原因。**</font>
   - solution：<font color=green>MC $\epsilon$-greedy算法</font>

### III. MC $\epsilon$-greedy

#### III.1 soft policy

- **定义**：如果一个策略在任意状态states下，采取任意action的概率都是正值，则称该策略为soft policy。
- **特征**：如果策略是soft policy, 那么单个episode只要足够长，就能visit every state-action pair many times.
- **价值**：此时，就不用以不同的state-action为起点生成大量的episodes了。也就自然满足了MC basic和MC exploring start两个算法中需要的exploring-starts条件。

#### III.2 $\epsilon$-greedy policy

- 策略内容：
$$\pi(a|s)\left\{\begin{matrix}
1-\epsilon , & for\ a_{greedy}=\underset{a\in \mathcal{A}(s)}{argmax\hat Q}(s, a) \\
\epsilon,   & for\ all\ a \in \mathcal{A}(s)
\end{matrix}\right. \ \ , 其中0<\epsilon \le 1$$

- 另一种等价的写法：
$$\pi(a|s)\left\{\begin{matrix}
1-\epsilon + \frac{\epsilon }{|\mathcal{A}|} , & for\ a_{greedy}=\underset{a\in \mathcal{A}(s)}{argmax\hat Q}(s, a) \\
\frac{\epsilon}{|\mathcal{A}|},   & for\ each\ a \in \mathcal{A}(s)，except\ a_{greedy}
\end{matrix}\right. \ \ , 其中0<\epsilon \le 1$$

- 当$\epsilon=0$时，变成greedy算法；当$\epsilon=1$，变成所有action之间的等概率抽样。

- 实现$\epsilon$-greedy抽样的方法：first generate a random number x in [0, 1] by following a uniform
distribution. If x ≥ $\epsilon$ , then we select the greedy action. If x < $\epsilon$, then we randomly select an action in A(s)

#### II.2 基于$\epsilon$-greedy实现MC算法

- 将$\epsilon$整合进MC的方法：
  - 将policy improvement step中的greedy改成$\epsilon$-greedy

- 算法伪码：
  - <img src="./pics/e_greedy.png" alt="alt text" width="560"/>

#### II.3 $\epsilon$-greedy中的exploration和exploitation

- $\epsilon$-greedy方法提供了一种平衡exploration和exploitation的方法。$\epsilon$的值越大，模型的exploration程度越高，反之exploitation程度就越高。

- 在实际算法使用中，$\epsilon$的值通常先高后低。早期多做exploration，然后逐步增加exploitation的比例。